# Rent Estimate API Walkthrough

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera, arielherrera@analyticsariel.com                                                      |
| External References   | <a href="https://rapidapi.com/moneals/api/rent-estimate" target="_blank">Rental API</a>|
| Input Datasets        | Rental Estimate Data |
| Output Datasets       | Json    |
| Input Data Source     | API |
| Output Data Source    | Json |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 20th May 2020 | Ariel Herrera | Notebook created to get data on rental comparables. |

## Other Details
This Notebook is a prototype.

In [2]:
import pandas as pd
import requests

## Functions

In [4]:
def get_api_key(api_key_id = "Realtor"):
  """
  Get the api key for website accessing.

  Table of key type and key value for privacy.

  Parameters
  ----------
  @api_key_id [string]: Key value in dataframe

  Returns
  -------
  [string]: API Key

  """
  # load api keys file
  df_api_keys = spark.read.format('csv').options(header='true', inferSchema='true').load('FileStore/tables/api_keys.csv').toPandas()
  
  # return api key if in dataset
  try:
    # get api key from id
    api_key = df_api_keys.loc[df_api_keys['Id'] == api_key_id]['Key'].iloc[0] # get key by id
    # return api key
    return api_key
  except IndexError:
    # get api key id list
    api_key_id_list = df_api_keys['Id'].unique().tolist()
    # print error message
    print('Cannot map key. Api key id must be one of the following options {0}'.format(api_key_id_list))

In [5]:
def api_rent_estimate(api_key, address, bedrooms, bathrooms, property_type=None, square_footage=None, comp_count=5, longitude=None, latitude=None, days_old=None):
    """
    Rent estimates by moneals.

    Call API to get rent estimates.

    Reference: https://rapidapi.com/moneals/api/rent-estimate/endpoints

    Parameters
    ----------
    @api_key [api_key]: Key to access data from API
    @address [string]: "Street Address, City, State"
    @bedrooms [string]: Number of bedrooms
    @bathrooms [string]: Number of bathrooms
    @property_type [string]: Options - Single Family, Apartment, Townhouse, Condo, Duplex-Triplex
    @square_footage [string]: Square footage of property
    @comp_count [string]: Number of comparable rentals to return, between 3 - 10
    @longitude [string]: Longitude of the property, can be provided in lieu of the address parameter
    @lattitude [string]: Lattitude of the property, can be provided in lieu of the address parameter
    @days_old [string]: Filters comparable rentals older than the provided value

    Returns
    -------
    [json] API response

    """

    # url for api
    url = "https://realtymole-rental-estimate-v1.p.rapidapi.com/rentalPrice"

    # enter parameters
    querystring = {
        "address":address,
        "bedrooms":bedrooms,
        "bathrooms":bathrooms,
        "propertyType": property_type,
        "squareFootage":square_footage,
        "compCount":comp_count,
        "longitude":longitude,
        "latitude":latitude
    }

    # header
    headers = {
      'x-rapidapi-host': "realtymole-rental-estimate-v1.p.rapidapi.com",
      'x-rapidapi-key': api_key
    }

    # wait for api to reset
    # time.sleep(1)
    
    # response
    response = requests.request("GET", url, headers=headers, params=querystring)

    return response.json()

## Variables

In [7]:
# api key to accessd ata
rapid_api_key = get_api_key(api_key_id = "Realtor")
location_str = "93 Winans Ave, Newark, New Jersey"

## Data: API Rental Comparables

In [9]:
import requests

url = "https://realtymole-rental-estimate-v1.p.rapidapi.com/rentalPrice"

querystring = {"address":location_str}

headers = {
    'x-rapidapi-host': "realtymole-rental-estimate-v1.p.rapidapi.com",
    'x-rapidapi-key': rapid_api_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

response.json()

Out[11]: {'rent': 1552.5,
 'rentRangeLow': 1234.97,
 'rentRangeHigh': 1870.03,
 'longitude': -74.1979727,
 'latitude': 40.7279272,
 'listings': [{'id': '95-Winans-Ave,-Newark,-NJ-07108',
 'formattedAddress': '95 Winans Ave, Newark, NJ 07108',
 'longitude': -74.198027,
 'latitude': 40.727837,
 'city': 'Newark',
 'state': 'NJ',
 'zipcode': '07108',
 'price': 850,
 'publishedDate': '2020-03-01T06:15:32.432Z',
 'distance': 0.011036474975433376,
 'daysOld': 80.28,
 'correlation': 0.9987,
 'address': '95 Winans Ave',
 'county': 'Essex County',
 'bedrooms': 3,
 'bathrooms': 1,
 'propertyType': 'Apartment'},
 {'id': '79-Winans-Ave,-Unit-1,-Newark,-NJ-07108',
 'formattedAddress': '79 Winans Ave, Unit 1, Newark, NJ 07108',
 'longitude': -74.19823,
 'latitude': 40.728421,
 'city': 'Newark',
 'state': 'NJ',
 'zipcode': '07108',
 'price': 1400,
 'publishedDate': '2020-01-18T00:06:37.106Z',
 'distance': 0.059099484128355376,
 'daysOld': 123.53,
 'correlation': 0.9958,
 'address': '79 Winans Ave, Unit 1',
 'county': 'Essex',
 'bedrooms': 3,
 'bathrooms': 1,
 'propertyType': 'Apartment'},
 {'id': '535-Bergen-Ave,-Unit-2,-Newark,-NJ-07108',
 'formattedAddress': '535 Bergen Ave, Unit 2, Newark, NJ 07108',
 'longitude': -74.198558,
 'latitude': 40.726067,
 'city': 'Newark',
 'state': 'NJ',
 'zipcode': '07108',
 'price': 1500,
 'publishedDate': '2020-05-20T04:59:04.437Z',
 'distance': 0.2128806970322293,
 'daysOld': 0.33,
 'correlation': 0.9894,
 'address': '535 Bergen Ave, Unit 2',
 'county': 'Essex County',
 'bedrooms': 3,
 'bathrooms': 1,
 'propertyType': 'Apartment'},
 {'id': '39-Winans-Ave,-Apt-2,-Newark,-NJ-07108',
 'formattedAddress': '39 Winans Ave, Apt 2, Newark, NJ 07108',
 'longitude': -74.199309,
 'latitude': 40.729648,
 'city': 'Newark',
 'state': 'NJ',
 'zipcode': '07108',
 'price': 1500,
 'publishedDate': '2020-03-25T05:55:18.472Z',
 'distance': 0.22226520506495992,
 'daysOld': 56.29,
 'correlation': 0.9883,
 'address': '39 Winans Ave, Apt 2',
 'county': 'Essex County',
 'bedrooms': 3,
 'bathrooms': 1,
 'propertyType': 'Apartment'},
 {'id': '4-Dandridge-Dr,-Newark,-NJ-07108',
 'formattedAddress': '4 Dandridge Dr, Newark, NJ 07108',
 'longitude': -74.197532,
 'latitude': 40.725572,
 'city': 'Newark',
 'state': 'NJ',
 'zipcode': '07108',
 'price': 1950,
 'publishedDate': '2020-05-02T17:58:05.740Z',
 'distance': 0.26480099365220855,
 'daysOld': 17.79,
 'correlation': 0.9866,
 'address': '4 Dandridge Dr',
 'county': 'Essex County',
 'bedrooms': 3,
 'bathrooms': 2.5,
 'propertyType': 'Single Family'},
 {'id': '10-Dandridge-Dr,-Newark,-NJ-07108',
 'formattedAddress': '10 Dandridge Dr, Newark, NJ 07108',
 'longitude': -74.197473,
 'latitude': 40.725714,
 'city': 'Newark',
 'state': 'NJ',
 'zipcode': '07108',
 'price': 1850,
 'publishedDate': '2020-01-08T21:40:34.874Z',
 'distance': 0.24995129506424854,
 'daysOld': 132.63,
 'correlation': 0.9862,
 'address': '10 Dandridge Dr',
 'county': 'Essex',
 'bedrooms': 3,
 'bathrooms': 2.5,
 'propertyType': 'Townhouse'},
 {'id': '18-Dandridge-Dr,-Newark,-NJ-07108',
 'formattedAddress': '18 Dandridge Dr, Newark, NJ 07108',
 'longitude': -74.197592,
 'latitude': 40.725429,
 'city': 'Newark',
 'state': 'NJ',
 'zipcode': '07108',
 'price': 1900,
 'publishedDate': '2020-05-20T04:59:04.426Z',
 'distance': 0.2799450736796103,
 'daysOld': 0.33,
 'correlation': 0.986,
 'address': '18 Dandridge Dr',
 'county': 'Essex County',
 'bedrooms': 4,
 'bathrooms': 2.5,
 'propertyType': 'Single Family'},
 {'id': '585-Bergen-Ave,-Unit-3,-Newark,-NJ-07108',
 'formattedAddress': '585 Bergen Ave, Unit 3, Newark, NJ 07108',
 'longitude': -74.198558,
 'latitude': 40.726067,
 'city': 'Newark',
 'state': 'NJ',
 'zipcode': '07108',
 'price': 1350,
 'publishedDate': '2019-04-25T15:31:41.648Z',
 'distance': 0.2128806970322293,
 'daysOld': 390.89,
 'correlation': 0.9855,
 'address': '585 Bergen Ave, Unit 3',
 'county': 'Essex',
 'bedrooms': 3,
 'bathrooms': 1,
 'propertyType': 'Apartment'},
 {'id': '585-Bergen-Ave,-Unit-1,-Newark,

# End Notebook